In [13]:
import tkinter as tk
from tkinter import simpledialog, messagebox
import csv
import os

class TodoList:
    def __init__(self, filename):
        self.filename = filename
        self.tasks = []
        self.load_tasks()

    def load_tasks(self):
        if os.path.exists(self.filename):
            with open(self.filename, 'r', newline='') as f:
                reader = csv.DictReader(f)
                self.tasks = list(reader)

    def save_tasks(self):
        with open(self.filename, 'w', newline='') as f:
            fieldnames = ['id', 'description', 'completed']
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            for task in self.tasks:
                writer.writerow(task)

    def add_task(self, task_description):
        task = {'id': len(self.tasks) + 1, 'description': task_description, 'completed': 'False'}
        self.tasks.append(task)
        self.save_tasks()

    def view_tasks(self):
        if not self.tasks:
            return "No tasks."
        else:
            task_list = "\n".join(f"{task['id']}. {task['description']} {'✓' if task['completed'] == 'True' else ''}" for task in self.tasks)
            return task_list

    def update_task(self, task_index, updated_task):
        if 1 <= task_index <= len(self.tasks):
            self.tasks[task_index - 1] = updated_task
            self.save_tasks()
            return f"Task {task_index} updated."
        else:
            return "Invalid task index."

    def mark_completed(self, task_index):
        if 1 <= task_index <= len(self.tasks):
            self.tasks[task_index - 1]['completed'] = "True"
            self.save_tasks()
            return f"Task {task_index} marked as completed."
        else:
            return "Invalid task index."

    def delete_task(self, task_index):
        if 1 <= task_index <= len(self.tasks):
            del self.tasks[task_index - 1]
            self.update_task_ids()
            self.save_tasks()
            return f"Task {task_index} deleted."
        else:
            return "Invalid task index."

    def clear_tasks(self):
        self.tasks = []
        self.save_tasks()

    def update_task_ids(self):
        for index, task in enumerate(self.tasks, start=1):
            task['id'] = index

class TodoListApp:
    def __init__(self, root):
        self.root = root
        self.root.title("To-Do List App")
        self.root.geometry("800x500")
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)  # Bind close event

        self.todo_list = TodoList("tasks.csv")

        # Frame for task list and controls
        frame = tk.Frame(self.root)
        frame.pack(padx=20, pady=20)

        # Task Listbox
        self.task_listbox = tk.Listbox(frame, width=70, height=15, font=("Arial", 12))
        self.task_listbox.grid(row=0, column=0, rowspan=5, padx=10)

        # Scrollbar for Task Listbox
        scrollbar = tk.Scrollbar(frame, orient=tk.VERTICAL, command=self.task_listbox.yview)
        scrollbar.grid(row=0, column=1, rowspan=5, sticky='ns')
        self.task_listbox.config(yscrollcommand=scrollbar.set)

        # Buttons
        button_style = {"padx": 10, "pady": 5, "font": ("Arial", 12)}

        add_task_button = tk.Button(frame, text="Add Task", command=self.add_task, bg="#4CAF50", fg="white", **button_style)
        add_task_button.grid(row=0, column=2, padx=10, pady=5, sticky='ew')

        edit_task_button = tk.Button(frame, text="Edit Task", command=self.edit_task, bg="#2196F3", fg="white", **button_style)
        edit_task_button.grid(row=1, column=2, padx=10, pady=5, sticky='ew')

        mark_complete_button = tk.Button(frame, text="Mark Completed", command=self.mark_completed, bg="#FFC107", **button_style)
        mark_complete_button.grid(row=2, column=2, padx=10, pady=5, sticky='ew')

        delete_task_button = tk.Button(frame, text="Delete Task", command=self.delete_task, bg="#f44336", fg="white", **button_style)
        delete_task_button.grid(row=3, column=2, padx=10, pady=5, sticky='ew')

        clear_tasks_button = tk.Button(frame, text="Clear All", command=self.clear_tasks, bg="#607D8B", fg="white", **button_style)
        clear_tasks_button.grid(row=4, column=2, padx=10, pady=5, sticky='ew')

        # Exit Button
        exit_button = tk.Button(self.root, text="Exit", command=self.on_closing, bg="#9E9E9E", **button_style)
        exit_button.pack(pady=10)

        # Refresh tasks initially
        self.refresh_tasks()

    def refresh_tasks(self):
        self.task_listbox.delete(0, tk.END)
        tasks = self.todo_list.view_tasks()
        if tasks == "No tasks.":
            self.task_listbox.insert(tk.END, "No tasks.")
        else:
            task_lines = tasks.split("\n")
            for task in task_lines:
                self.task_listbox.insert(tk.END, task)

    def add_task(self):
        task_description = simpledialog.askstring("Add Task", "Enter task description:")
        if task_description:
            self.todo_list.add_task(task_description)
            messagebox.showinfo("Task Added", "Task added successfully.")
            self.refresh_tasks()

    def edit_task(self):
        task_index_str = simpledialog.askstring("Edit Task", "Enter task index to edit:")
        try:
            task_index = int(task_index_str)
            current_task_description = self.todo_list.tasks[task_index - 1]['description']
            new_task_description = simpledialog.askstring("Edit Task", f"Current task: {current_task_description}\nEnter new task description:")
            if new_task_description:
                updated_task = {'id': task_index, 'description': new_task_description, 'completed': self.todo_list.tasks[task_index - 1]['completed']}
                result = self.todo_list.update_task(task_index, updated_task)
                messagebox.showinfo("Task Updated", result)
                self.refresh_tasks()
        except (ValueError, IndexError):
            messagebox.showerror("Error", "Please enter a valid task index.")

    def mark_completed(self):
        task_index_str = simpledialog.askstring("Mark Completed", "Enter task index to mark as completed:")
        try:
            task_index = int(task_index_str)
            result = self.todo_list.mark_completed(task_index)
            messagebox.showinfo("Mark Completed", result)
            self.refresh_tasks()
        except (ValueError, IndexError):
            messagebox.showerror("Error", "Please enter a valid task index.")

    def delete_task(self):
        task_index_str = simpledialog.askstring("Delete Task", "Enter task index to delete:")
        try:
            task_index = int(task_index_str)
            result = self.todo_list.delete_task(task_index)
            messagebox.showinfo("Task Deleted", result)
            self.refresh_tasks()
        except (ValueError, IndexError):
            messagebox.showerror("Error", "Please enter a valid task index.")

    def clear_tasks(self):
        confirmation = messagebox.askyesno("Clear All Tasks", "Are you sure you want to clear all tasks?")
        if confirmation:
            self.todo_list.clear_tasks()
            messagebox.showinfo("Tasks Cleared", "All tasks cleared successfully.")
            self.refresh_tasks()

    def on_closing(self):
        self.todo_list.save_tasks() 
        self.root.destroy()

def main():
    root = tk.Tk()
    app = TodoListApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()
